## The [Architecture of Complexity] by Herbert A. Simon

There once were two watchmakers, named Hora and Tempus, who manufactured very
fine watches. Both of them were highly regarded, and the phones in their
workshops rang frequently -new customers were constantly calling them. However,
Hora prospered, while Tempus became poorer and poorer and finally lost his shop.
What was the reason?

The watches the men made consisted of about 1,000 parts each. Tempus had so
constructed his that if he had one partly assembled and had to put it down - to
answer the phone say - it immediately fell to pieces and had to be reassembled
from the elements. The better the customers liked his watches, the more they
phoned him, the more difficult it became for him to find enough uninterrupted
time to finish a watch.

The watches that Hora made were no less complex than those of Tempus. But he had
designed them so that he could put together subassemblies of about ten elements
each. Ten of these subassemblies, again, could be put together into a larger
subassembly; and a system of ten of the latter subassemblies constituted the
whole watch. Hence, when Hora had to put down a partly assembled watch in order
to answer the phone, he lost only a small part of his work, and he assembled his
watches in only a fraction of the man-hours it took Tempus. 

[Architecture of Complexity]: https://www2.econ.iastate.edu/tesfatsi/ArchitectureOfComplexity.HSimon1962.pdf

In [1]:
import sys
sys.path.append("../")


In [2]:
import asyncio


async def test_wait(timeout=2, seconds_passed=0):
    await asyncio.sleep(timeout)
    seconds_passed += timeout
    return "done"

sequence = ["one", "two", "three"]


async def test(sequence=sequence):
    seconds_passed = 0
    dones = []
    for step in sequence:
        print(f"Starting {step}")
        result = await test_wait(1, seconds_passed)
        print("\n")
        print(f"Finished {step}")
        dones.append(result)
    return dones

# response = await test()
# print(response)

In [8]:
import asyncio
import json

from functools import partial, wraps

from fns.intent import catalog, get_intent
from tools.is_done import is_done


def async_wrap(func):
    @wraps(func)
    async def run(*args, loop=None, executor=None, **kwargs):
        if loop is None:
            loop = asyncio.get_event_loop()
        pfunc = partial(func, *args, **kwargs)
        return await loop.run_in_executor(executor, pfunc)
    return run


async def spool(
    history,
    max_links=3,
    link=0,
    previous_intents=[]
):
    """

    Spins up an interactive and asynchronous communication channel with the
    following services:

    - Intent Processor
    - Function Payloader
    - Actions (Functions to Call)
    - IsDone (Determines if the conversation is done)

    """
    # print(f"Link # {link}")
    if link >= max_links:
        return history
    else:
        try:
            # get the first user message
            first_user_msg = next(
                (h["content"] for h in history if h["role"] == "user"),
                None
            )
            # print(f"First User Message: {first_user_msg}")

            last_user_msg = next(
                (h["content"]
                 for h in reversed(history) if h["role"] == "user"),
                None
            )
            # print(f"Last User Message: {last_user_msg}")

            last_asst_msg = next(
                (h["content"]
                 for h in reversed(history) if h["role"] == "assistant"),
                None
            )
            # print(f"Last Assistant Message: {last_asst_msg}")

            # get the intent for the message
            intent = get_intent(last_user_msg)
            # print(f"Intent: {intent}")

            fn, fu, id = intent["function"], intent["follow_up"], intent["name"]
            # print(f"Function: {fn}")
            # print(f"ID: {id}")
            # convert func to async so we can await it
            func = async_wrap(fn)

            if id in previous_intents:
                print(f"🔥 Already did this intent: {id}")
                return history
            else:
                previous_intents.append(id)
            print(f"\nExectuting tool: `{id}`")
            if fu: 
                print(f"Follow Up: {fu}") 
            else:
                print("I have what I need. Wrapping up...")

            # This needs to be handled by the payloader in the func(history)
            if id == "elaborate_vtx_url":
                call = await func(last_asst_msg)
            elif id == "llm_pick":
                call = await func(first_user_msg, last_asst_msg)
            elif id == "llm_eloquent":
                call = await func(f"Query: {first_user_msg}\n\n Content:{last_asst_msg}")
            else:
                call = await func(last_user_msg)

            history.append({
                "role": "assistant",
                "content": json.dumps(call) if type(call) != str else call
            })

            if fu:
                history.append({
                    "role": "user",
                    "content": fu
                })
                # print("🧵 Spooling...")
                return await spool(
                    history,
                    max_links,
                    link + 1,
                    previous_intents
                )
            else:
                print(f"\nResult for query: {first_user_msg}\n")
                print(call)
                return history
        except Exception as e:
            print(f"🔥 Error: {e}")
            return history

In [9]:
test_history = [
    # {"role": "user", "content": "What products does Vertex have?"},
    {"role": "user", "content": "How do I contact Vertex Inc customer service?"},
]

In [10]:
result = await spool(test_history, 5)


Exectuting tool: `search_vtx_com`
Follow Up: The results are a list of pages. I will pick the most relevant one for you.

Exectuting tool: `llm_pick`
Follow Up: The selected URL needs additional context. I will elaborate on it.

Exectuting tool: `elaborate_vtx_url`
Follow Up: Text is verbose, I will summarize it for you.

Exectuting tool: `llm_eloquent`
I have what I need. Wrapping up...

Result for query: How do I contact Vertex Inc customer service?

# Vertex Inc Customer Service Contact Information

- **Product Support**
  - Self-help support: Visit Vertex Community (myVertex)
  - North America Phone: 1-800-281-1900
  - International Phone: +1-610-640-4210
  - For Vertex Cloud support:
    - Phone: 1-855-221-5885
    - Email: cloudsupport@vertexinc.com

## Training

Visit Training for course registration.
  
   - North America Phone: 1-877-455-2645  
   - International Phone: +1-484-595-2590
  
## General Inquiries
  
Fill out the form on the website to get in touch with Account Ma

In [11]:
#print(json.dumps(result, indent=2))

In [12]:
import re

from IPython.display import Markdown, display

final_response = result[-1]["content"]
display(Markdown(final_response))

# Vertex Inc Customer Service Contact Information

- **Product Support**
  - Self-help support: Visit Vertex Community (myVertex)
  - North America Phone: 1-800-281-1900
  - International Phone: +1-610-640-4210
  - For Vertex Cloud support:
    - Phone: 1-855-221-5885
    - Email: cloudsupport@vertexinc.com

## Training

Visit Training for course registration.
  
   - North America Phone: 1-877-455-2645  
   - International Phone: +1-484-595-2590
  
## General Inquiries
  
Fill out the form on the website to get in touch with Account Management, Customer Support, Training, Sales or other departments.

## Global Locations
  
### Headquarters
  
**Philadelphia**
  
Phone: 1-(800)-3553500
Address:
2301 Renaissance Blvd,
King of Prussia, PA,
19406 United States.

**London**

Phone:+44 (0)20.3906.7630 
Address:
30 Furnival Street,
3rd Floor,
London EC4A. 

### Other Locations

**Naperville**

Phone :+16303282600 
Address :
40 Shuman Boulevard Suite160 Naperville IL60563 United States.

**São Paulo**

Phone :+55(11)50953433 
Address :
Regus EZ Tower – Torre B Rua Arquiteto Olavo Redig de Campos105 –24o Andar Chácara Santo Antônio(Zona Sul) São Paulo SP04718 Brazil.

**Stockholm**

Phone :+46704977428 
Address :
VGTS Sweden AB c/o Hummelklappen Villagatan19 Stockholm11432 Sweden.

**Chennai**

Level6 Chennai Citi Centre No10/11 Dr Radhakrishnan Salai Mylapore Chennai600004 India.

For more information about their services and products visit [Vertex Inc](https://www.vertexinc.com/).